## Retriever And Chain With Langchain 

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY') 

## Loading docs

In [21]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('imagepro.pdf')
docs=loader.load()
docs

[Document(page_content='GLOBAL\n \nEDITION\nThis is a special edition of an established \ntitle widely used by colleges and universities \nthroughout the world. Pearson published this \nexclusive edition for the benefit of students \noutside the United States and Canada. If \nyou purchased this book within the United \nStates or Canada, you should be aware that \nit has been imported without the approval of \nthe Publisher or Author. The Global Edition \nis not supported in the United States and \nCanada.\nPearson Global Edition\nGLOBAL\n \nEDITION\nFor these Global Editions, the editorial team at Pearson has \ncollaborated with educators across the world to address a \nwide range of subjects and requirements, equipping students \nwith the best possible learning tools. This Global Edition \npreserves the cutting-edge approach and pedagogy of the \noriginal, but also features alterations, customization, and \nadaptation from the North American version.\nD\nigital Image Processing\n FOUR

## Splitting docs

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='GLOBAL\n \nEDITION\nThis is a special edition of an established \ntitle widely used by colleges and universities \nthroughout the world. Pearson published this \nexclusive edition for the benefit of students \noutside the United States and Canada. If \nyou purchased this book within the United \nStates or Canada, you should be aware that \nit has been imported without the approval of \nthe Publisher or Author. The Global Edition \nis not supported in the United States and \nCanada.\nPearson Global Edition\nGLOBAL\n \nEDITION\nFor these Global Editions, the editorial team at Pearson has \ncollaborated with educators across the world to address a \nwide range of subjects and requirements, equipping students \nwith the best possible learning tools. This Global Edition \npreserves the cutting-edge approach and pedagogy of the \noriginal, but also features alterations, customization, and \nadaptation from the North American version.\nD\nigital Image Processing\n FOUR

In [23]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='GLOBAL\n \nEDITION\nThis is a special edition of an established \ntitle widely used by colleges and universities \nthroughout the world. Pearson published this \nexclusive edition for the benefit of students \noutside the United States and Canada. If \nyou purchased this book within the United \nStates or Canada, you should be aware that \nit has been imported without the approval of \nthe Publisher or Author. The Global Edition \nis not supported in the United States and \nCanada.\nPearson Global Edition\nGLOBAL\n \nEDITION\nFor these Global Editions, the editorial team at Pearson has \ncollaborated with educators across the world to address a \nwide range of subjects and requirements, equipping students \nwith the best possible learning tools. This Global Edition \npreserves the cutting-edge approach and pedagogy of the \noriginal, but also features alterations, customization, and \nadaptation from the North American version.\nD\nigital Image Processing\n FOUR

## Vector Store


In [24]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30], OpenAIEmbeddings())

In [25]:
db

In [26]:
query="Department of Agriculture and Farmers Welfare"
result=db.similarity_search(query)
result[0]

Document(page_content='Roger Heady, Juan A. Herrera, John M. Hudak, Michael Hurwitz, Chris J. Johannsen, \nRhonda Knighton, Don P . Mitchell, A. Morris, Curtis C. Ober, David. R. Pickens, \nMichael Robinson, Michael Shaffer, Pete Sites, Sally Stowe, Craig Watson, David \nK. Wehe, and Robert A. West. We also wish to acknowledge other individuals and \norganizations cited in the captions of numerous ﬁgures throughout the book for \ntheir permission to use that material. \nWe also thank Scott Disanno, Michelle Bayman, Rose Kernan, and Julie Bai for \ntheir support and signiﬁcant patience during the production of the book.\nR.C.G.\nR.E.W.\nDIP4E_GLOBAL_Print_Ready.indb   12\n6/16/2017   2:01:57 PM\nwww.EBooksWorld.ir', metadata={'source': 'imagepro.pdf', 'page': 13})

## LLM model using 

In [27]:
from langchain_community.llms import Ollama
## Load Ollama LLama-3 LLM Model
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

## Design ChatPrompt Template

In [28]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step bt step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>                                          
Question: {input}""")

## Chain Introduction || Create Stuff Document Chain

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain


document_chain=create_stuff_documents_chain(llm,prompt)

## Retrievers
- A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.
[Learn more](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/)

In [30]:
retriever= db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002122EAE49D0>)

## Retrievers Chains
- Retrievers chain: This chain takes in user inquiry, which is then passed to the retriver to fetch relevant documents. Those documents(and original inputs) are then passed to an LLM to generate a response

In [31]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

## Invoking Any Query now

In [36]:
response=retrieval_chain.invoke({"input":"""image processing with an introduction to techniques for image 
pattern classification"""})

In [37]:
print(response['answer'])

Based on the provided context, I would answer your question as follows:

The companion website www.ImageProcessingPlace.com offers additional support in a number of important areas. 

For the Student or Independent Reader, the site contains tutorials on topics relevant to the material in the book. One specific tutorial that could be helpful for image pattern classification is found in the Tutorials section.

This edition of Digital Image Processing is a major revision of the book and continues to provide an introduction to basic concepts and methodologies applicable to digital image processing. To achieve these objectives, the authors focused on material that they believe is fundamental and whose scope of application is not limited to the solution of specialized problems.

One possible area for exploring image pattern classification could be in the Wavelet and Other Image Transforms chapter (Chapter 7) where various transforms such as Fourier-Related Transforms, Walsh-Hadamard Transfor